In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd

# 🧱 Agent: Train the model for multiple tokens, including sentiment-weighted total
def train_multiple_tokens_models_with_sentiment(all_tokens_data):
    # Dictionary to store the trained models for each token
    all_models = {}

    # Loop through each token's data
    for token, combined_df in all_tokens_data.items():
        print(f"Training model for {token}...")

        # Feature selection (including sentiment-weighted total)
        X = combined_df[['value_x', 'value_y', 'return', 'rolling_avg', 'value']]  # Features
        y = combined_df['buy_signal'].astype(int)  # Target: buy_signal (binary classification)

        # Split the data into training and testing sets (80% train, 20% test)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Create and train the XGBoost classifier
        model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')  # Avoid label encoder warning
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)

        print(f"Accuracy for {token}: {accuracy:.4f}")
        print(f"Confusion Matrix for {token}:")
        print(conf_matrix)

        # Feature importance (optional)
        xgb.plot_importance(model)

        # Store the trained model in the dictionary
        all_models[token] = model

    return all_models
